In [1]:
# _*_ coding: utf-8 _*_
# @time : 2022/1/12  16:29
# @name : traditionalMethod.py
# @author : 霜晨月~
import cv2
import numpy as np


# 1.1、帧差分法-->二帧法
def dis_2frame():
    cap = cv2.VideoCapture(0)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    last_frame = np.zeros((height, width), dtype=np.uint8)  # 前1帧
    num = 0
    if not cap.isOpened():
        print('Error opening video or file!')
    while cap.isOpened():  # 视频打开成功
        ret, frame = cap.read()
        if ret:
            # 当前帧
            tempFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # 差分帧
            disFrame = cv2.absdiff(tempFrame, last_frame)
            # 中值滤波去噪
            median = cv2.medianBlur(disFrame, 3)
            # 二值化
            ret, threshold_frame = cv2.threshold(median, 20, 255, cv2.THRESH_BINARY)
            # 高斯模糊
            gauss_image = cv2.GaussianBlur(threshold_frame, (3, 3), 0)
            cv2.imshow('dis_medianBlur_threshold_gaussianBlur', gauss_image)
            # 更新，便于下一次差分
            last_frame = tempFrame
            num += 1
            if cv2.waitKey(20) & 0xFF == 27:
                cv2.destroyAllWindows()
                break
    cap.release()

# 1.2、帧差分法-->三帧法：12相减，23相减，与运算
def dis_3frame():
    cap = cv2.VideoCapture(0)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame1 = np.zeros((height, width), dtype=np.uint8)  # 前2帧 1
    frame2 = frame1  # 前1帧 2
    num = 0
    if not cap.isOpened():
        print('Error opening video or file!')

    while cap.isOpened():  # 视频打开成功
        ret, frame = cap.read()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if ret:
            # 当前帧
            temp_frame = frame_gray
            # 12差分
            dis_frame1 = cv2.absdiff(frame2, frame1)
            _, thresh1 = cv2.threshold(dis_frame1, 40, 255, cv2.THRESH_BINARY)  # 二值，大于40的为255，小于0
            # 23差分
            dis_frame2 = cv2.absdiff(temp_frame, frame2)
            _, thresh2 = cv2.threshold(dis_frame2, 40, 255, cv2.THRESH_BINARY)  # 二值，大于40的为255，小于0
            # 与运算
            dis_frame = cv2.bitwise_and(thresh1, thresh2)  # 二值化图像
            # 形态学去噪
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
            erode = cv2.erode(dis_frame, kernel)  # 腐蚀
            # dilate = cv2.dilate(erode, kernel)  # 膨胀
            # dilate = cv2.dilate(dilate, kernel)  # 膨胀

            img, contours, hei = cv2.findContours(erode.copy(), mode=cv2.RETR_EXTERNAL,
                                                  method=cv2.CHAIN_APPROX_SIMPLE)  # 寻找轮廓
            for contour in contours:
                if 100 < cv2.contourArea(contour) < 40000:
                    x, y, w, h = cv2.boundingRect(contour)  # 找方框
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255))
            cv2.namedWindow("binary", cv2.WINDOW_NORMAL)
            cv2.namedWindow("dilate", cv2.WINDOW_NORMAL)
            cv2.namedWindow("frame", cv2.WINDOW_NORMAL)
            cv2.imshow("binary", dis_frame)
            cv2.imshow("dilate", erode)
            cv2.imshow("frame", frame)
            # 更新，便于下一次差分
            frame1, frame2 = frame2, temp_frame
            num += 1
            if cv2.waitKey(20) & 0xFF == 27:
                cv2.destroyAllWindows()
                break
    cap.release()


# 2、背景减除法
def createBackground():
    cap = cv2.VideoCapture(0)
    # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    # fgbg = cv2.bgsegm.createBackgroundSubtractorGMG()
    fgbg = cv2.createBackgroundSubtractorMOG2()
    while cap.isOpened():
        ret, frame = cap.read()

        fgmask = fgbg.apply(frame)
        # fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
        cv2.imshow('fgmask', fgmask)
        if cv2.waitKey(20) & 0xFF == 27:
            cv2.destroyAllWindows()
            break
    cap.release()


# 3、光流法
def lightFlow():
    cap = cv2.VideoCapture(0)
    feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)  # ShiTomasi 角点检测参数
    ret, frame0 = cap.read()
    w, h = cap.get(3), cap.get(4)
    FPS = cap.get(5)
    print('size:', w, h)
    print('FPS:', FPS)
    frame0_gray = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(frame0_gray, mask=None, **feature_params)  # **param 关键字参数

    mask = np.zeros_like(frame0)  # 创建一个蒙版用来画轨迹,i.e.和每帧图像大小相同的全0张量
    lk_params = dict(winSize=(15, 15), maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))  # lucas kanade光流法参数
    color = np.random.randint(0, 255, (100, 3))  # 创建随机颜色
    while cap.isOpened():
        ret, frame = cap.read()
        tempFrame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        p1, st, err = cv2.calcOpticalFlowPyrLK(frame0_gray, tempFrame_gray, p0, None, **lk_params)
        # 选取好的跟踪点
        good2track_temp = p1[st == 1]
        good2track_0 = p0[st == 1]

        # 画出轨迹
        for i, (new, old) in enumerate(zip(good2track_temp, good2track_0)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)  # 添加了该帧光流的轨迹图
            frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
        img = cv2.add(frame, mask)  # 将该图和轨迹图合并
        cv2.imshow('frame', img)

        if cv2.waitKey(20) & 0xFF == 27:
            cv2.destroyAllWindows()
            break
        # 更新上一帧的图像和追踪点
        old_gray = tempFrame_gray.copy()
        p0 = good2track_0.reshape(-1, 1, 2)
    cap.release()


if __name__ == '__main__':
    dis_2frame()
    # dis_3frame()
    # createBackground()
    # lightFlow()


ModuleNotFoundError: No module named 'cv2'